# Feature Engineering Pipeline - Black Friday Sales

**Cel:** Stworzenie zaawansowanych cech dla modeli Machine Learning

**Pipeline:**
1. Ładowanie danych surowych
2. Feature Engineering:
   - Agregacje per użytkownik
   - Agregacje per produkt
   - RFM Features
   - Categorical Encoding
   - Interakcje cech
3. Zapis do Delta Lake

---

In [ ]:
# Imports
import sys
sys.path.append('../')

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, sum as _sum, avg, min as _min, max as _max,
    countDistinct, stddev, when, lit, dense_rank, percent_rank,
    broadcast, concat_ws, row_number
)
from pyspark.sql.window import Window
from pyspark.ml.feature import (
    StringIndexer, OneHotEncoder, VectorAssembler,
    StandardScaler, MinMaxScaler
)
from pyspark.ml import Pipeline

from config.spark_config import SparkConfig

import warnings
warnings.filterwarnings('ignore')

## 1. Inicjalizacja Spark Session

In [ ]:
# Start Spark Session
spark = SparkConfig.get_spark_session("BlackFriday-FeatureEngineering")

print("\n✓ Spark session ready!")
print(f"Spark UI: http://localhost:4040")

## 2. Ładowanie Danych

In [ ]:
# Load raw data
df_raw = spark.read.csv(
    "../data/raw/BlackFriday.csv",
    header=True,
    inferSchema=True
)

print(f"\nDataset shape: {df_raw.count():,} rows x {len(df_raw.columns)} columns")
df_raw.printSchema()

In [ ]:
# Show sample
df_raw.show(5, truncate=False)

## 3. Data Quality & Cleansing

In [ ]:
# Check for duplicates
total_rows = df_raw.count()
unique_rows = df_raw.dropDuplicates().count()

print(f"Total rows: {total_rows:,}")
print(f"Unique rows: {unique_rows:,}")
print(f"Duplicates: {total_rows - unique_rows:,}")

In [ ]:
# Check missing values
from pyspark.sql.functions import isnan, isnull

missing_counts = df_raw.select([
    count(when(col(c).isNull(), c)).alias(c) for c in df_raw.columns
])

print("\nMissing values per column:")
missing_counts.show()

In [ ]:
# Clean data
df_clean = df_raw.dropDuplicates()

# Fill missing Product_Category_2 and Product_Category_3 with 0 (indicating no secondary category)
df_clean = df_clean.fillna({
    'Product_Category_2': 0,
    'Product_Category_3': 0
})

print(f"\n✓ Cleaned data: {df_clean.count():,} rows")

## 4. Feature Engineering - User-Level Aggregations

In [ ]:
# User aggregations
user_features = df_clean.groupBy("User_ID").agg(
    # Purchase statistics
    count("*").alias("user_purchase_count"),
    _sum("Purchase").alias("user_total_spent"),
    avg("Purchase").alias("user_avg_purchase"),
    _min("Purchase").alias("user_min_purchase"),
    _max("Purchase").alias("user_max_purchase"),
    stddev("Purchase").alias("user_std_purchase"),
    
    # Product diversity
    countDistinct("Product_ID").alias("user_unique_products"),
    countDistinct("Product_Category_1").alias("user_unique_categories"),
    
    # Categorical modes (most frequent)
    # We'll take the first value as a proxy for mode
)

# Add derived features
user_features = user_features.withColumn(
    "user_purchase_range",
    col("user_max_purchase") - col("user_min_purchase")
)

user_features = user_features.withColumn(
    "user_avg_products_per_transaction",
    col("user_unique_products") / col("user_purchase_count")
)

print(f"\nUser features created: {user_features.count():,} users")
user_features.show(5)

## 5. Feature Engineering - Product-Level Aggregations

In [ ]:
# Product aggregations
product_features = df_clean.groupBy("Product_ID").agg(
    count("*").alias("product_purchase_count"),
    countDistinct("User_ID").alias("product_unique_users"),
    _sum("Purchase").alias("product_total_revenue"),
    avg("Purchase").alias("product_avg_price"),
    _min("Purchase").alias("product_min_price"),
    _max("Purchase").alias("product_max_price"),
    stddev("Purchase").alias("product_std_price")
)

# Product popularity ranking
window_spec = Window.orderBy(col("product_purchase_count").desc())
product_features = product_features.withColumn(
    "product_popularity_rank",
    dense_rank().over(window_spec)
)

# Normalize popularity to 0-1 scale
product_features = product_features.withColumn(
    "product_popularity_score",
    percent_rank().over(window_spec)
)

print(f"\nProduct features created: {product_features.count():,} products")
product_features.show(5)

## 6. Feature Engineering - Category-Level Aggregations

In [ ]:
# Category aggregations
category_features = df_clean.groupBy("Product_Category_1").agg(
    count("*").alias("category_purchase_count"),
    avg("Purchase").alias("category_avg_price"),
    _sum("Purchase").alias("category_total_revenue")
)

print(f"\nCategory features: {category_features.count()} categories")
category_features.orderBy(col("category_total_revenue").desc()).show(10)

## 7. RFM Features (Recency, Frequency, Monetary)

In [ ]:
# Create RFM features
# Note: Dataset doesn't have timestamps, so we'll create proxy RFM based on transaction ordering

# Frequency: Already have user_purchase_count
# Monetary: Already have user_total_spent
# Recency: Use row_number as proxy for time (last purchase)

window_user = Window.partitionBy("User_ID").orderBy(col("Purchase").desc())

df_with_recency = df_clean.withColumn(
    "transaction_order",
    row_number().over(window_user)
)

# Get recency (1 = most recent for each user)
recency_features = df_with_recency.filter(col("transaction_order") == 1).select(
    "User_ID",
    col("Purchase").alias("last_purchase_amount")
)

# Join recency with user features to create full RFM
rfm_features = user_features.join(
    recency_features,
    on="User_ID",
    how="left"
)

# RFM Scoring (quintiles)
rfm_features = rfm_features.withColumn(
    "rfm_frequency_score",
    percent_rank().over(Window.orderBy(col("user_purchase_count")))
)

rfm_features = rfm_features.withColumn(
    "rfm_monetary_score",
    percent_rank().over(Window.orderBy(col("user_total_spent")))
)

# Combined RFM score
rfm_features = rfm_features.withColumn(
    "rfm_score",
    (col("rfm_frequency_score") + col("rfm_monetary_score")) / 2
)

print("\nRFM Features:")
rfm_features.select(
    "User_ID", "user_purchase_count", "user_total_spent",
    "rfm_frequency_score", "rfm_monetary_score", "rfm_score"
).show(10)

## 8. Join All Features

In [ ]:
# Join user features back to main dataset
df_enriched = df_clean.join(
    broadcast(user_features),
    on="User_ID",
    how="left"
)

# Join product features
df_enriched = df_enriched.join(
    broadcast(product_features),
    on="Product_ID",
    how="left"
)

# Join category features
df_enriched = df_enriched.join(
    broadcast(category_features),
    on="Product_Category_1",
    how="left"
)

# Join RFM features
df_enriched = df_enriched.join(
    broadcast(rfm_features.select(
        "User_ID", "rfm_frequency_score", "rfm_monetary_score", "rfm_score"
    )),
    on="User_ID",
    how="left"
)

print(f"\n✓ Enriched dataset: {df_enriched.count():,} rows x {len(df_enriched.columns)} columns")
print("\nNew features added:")
print([c for c in df_enriched.columns if c not in df_clean.columns])

## 9. Interaction Features

In [ ]:
# Create interaction features
df_enriched = df_enriched.withColumn(
    "purchase_vs_user_avg_ratio",
    col("Purchase") / col("user_avg_purchase")
)

df_enriched = df_enriched.withColumn(
    "purchase_vs_product_avg_ratio",
    col("Purchase") / col("product_avg_price")
)

df_enriched = df_enriched.withColumn(
    "purchase_vs_category_avg_ratio",
    col("Purchase") / col("category_avg_price")
)

# Is this purchase above user's average?
df_enriched = df_enriched.withColumn(
    "is_above_user_avg",
    when(col("Purchase") > col("user_avg_purchase"), 1).otherwise(0)
)

# High value customer flag (top 20%)
df_enriched = df_enriched.withColumn(
    "is_high_value_customer",
    when(col("rfm_score") >= 0.8, 1).otherwise(0)
)

print("\n✓ Interaction features created")

## 10. Categorical Encoding

In [ ]:
# Define categorical columns
categorical_cols = [
    "Gender",
    "Age",
    "City_Category",
    "Stay_In_Current_City_Years"
]

# String Indexing
indexers = [
    StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="keep")
    for col in categorical_cols
]

# One-Hot Encoding
encoders = [
    OneHotEncoder(inputCol=col + "_index", outputCol=col + "_vec", dropLast=False)
    for col in categorical_cols
]

# Create pipeline
encoding_pipeline = Pipeline(stages=indexers + encoders)

# Fit and transform
encoding_model = encoding_pipeline.fit(df_enriched)
df_encoded = encoding_model.transform(df_enriched)

print("\n✓ Categorical encoding completed")
print(f"Total columns after encoding: {len(df_encoded.columns)}")

## 11. Feature Selection for ML

In [ ]:
# Select features for ML models
feature_cols = [
    # Categorical (indexed)
    "Gender_index", "Age_index", "City_Category_index",
    "Stay_In_Current_City_Years_index",
    
    # Numerical demographics
    "Occupation", "Marital_Status",
    
    # Product categories
    "Product_Category_1", "Product_Category_2", "Product_Category_3",
    
    # User features
    "user_purchase_count", "user_total_spent", "user_avg_purchase",
    "user_std_purchase", "user_unique_products", "user_unique_categories",
    
    # Product features
    "product_purchase_count", "product_unique_users", "product_avg_price",
    "product_popularity_score",
    
    # Category features
    "category_avg_price",
    
    # RFM features
    "rfm_frequency_score", "rfm_monetary_score", "rfm_score",
    
    # Interaction features
    "purchase_vs_user_avg_ratio", "purchase_vs_product_avg_ratio",
    "is_above_user_avg", "is_high_value_customer"
]

# Fill any remaining nulls with 0
df_encoded = df_encoded.fillna(0, subset=feature_cols)

# Vector Assembler
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features",
    handleInvalid="skip"
)

df_features = assembler.transform(df_encoded)

print(f"\n✓ Feature vector assembled with {len(feature_cols)} features")

## 12. Feature Scaling

In [ ]:
# Standard Scaler
scaler = StandardScaler(
    inputCol="features",
    outputCol="scaled_features",
    withMean=False,  # Sparse vectors don't support centering
    withStd=True
)

scaler_model = scaler.fit(df_features)
df_scaled = scaler_model.transform(df_features)

print("\n✓ Features scaled")

## 13. Final Feature Set Summary

In [ ]:
# Select final columns for ML
final_cols = [
    # IDs
    "User_ID", "Product_ID",
    
    # Target
    "Purchase",
    
    # Original features
    "Gender", "Age", "Occupation", "City_Category",
    "Stay_In_Current_City_Years", "Marital_Status",
    "Product_Category_1", "Product_Category_2", "Product_Category_3",
    
    # Engineered features (all)
] + [c for c in df_scaled.columns if c.startswith(("user_", "product_", "category_", "rfm_", "is_", "purchase_vs"))] + [
    # ML-ready features
    "features", "scaled_features"
]

df_final = df_scaled.select(*final_cols)

print("\n" + "="*60)
print("FEATURE ENGINEERING COMPLETE")
print("="*60)
print(f"Final dataset: {df_final.count():,} rows x {len(df_final.columns)} columns")
print(f"\nTotal features in vector: {len(feature_cols)}")
print("\nFeature categories:")
print(f"  - User features: {len([c for c in feature_cols if c.startswith('user_')])}")
print(f"  - Product features: {len([c for c in feature_cols if c.startswith('product_')])}")
print(f"  - Category features: {len([c for c in feature_cols if c.startswith('category_')])}")
print(f"  - RFM features: {len([c for c in feature_cols if c.startswith('rfm_')])}")
print(f"  - Interaction features: {len([c for c in feature_cols if 'ratio' in c or 'is_' in c])}")
print(f"  - Demographics: {len([c for c in feature_cols if c in categorical_cols or c in ['Occupation', 'Marital_Status']])}")

In [ ]:
# Show sample
df_final.select(
    "User_ID", "Product_ID", "Purchase",
    "user_avg_purchase", "product_popularity_score", "rfm_score"
).show(10)

## 14. Save to Delta Lake

In [ ]:
# Save to Delta Lake
delta_path = "../data/processed/delta/features"

df_final.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("City_Category", "Age") \
    .save(delta_path)

print(f"\n✓ Data saved to Delta Lake: {delta_path}")

In [ ]:
# Save individual feature sets for specific use cases

# 1. User-level features (for segmentation)
rfm_features.write \
    .format("delta") \
    .mode("overwrite") \
    .save("../data/processed/delta/user_features")

print("✓ User features saved")

# 2. Product-level features (for recommendations)
product_features.write \
    .format("delta") \
    .mode("overwrite") \
    .save("../data/processed/delta/product_features")

print("✓ Product features saved")

# 3. Encoding pipeline model
encoding_model.write().overwrite().save("../models/encoding_pipeline")

print("✓ Encoding pipeline saved")

# 4. Scaler model
scaler_model.write().overwrite().save("../models/scaler_model")

print("✓ Scaler model saved")

## 15. Validation & Statistics

In [ ]:
# Feature statistics
print("\n" + "="*60)
print("FEATURE STATISTICS")
print("="*60)

# Select numerical features for stats
numerical_features = [
    "Purchase",
    "user_avg_purchase", "user_purchase_count", "user_total_spent",
    "product_popularity_score", "product_avg_price",
    "rfm_score"
]

df_final.select(numerical_features).describe().show()

In [ ]:
# Check for nulls in final dataset
null_counts = df_final.select([
    count(when(col(c).isNull(), c)).alias(c) for c in df_final.columns[:20]  # First 20 cols
])

print("\nNull counts in final dataset (first 20 columns):")
null_counts.show()

In [ ]:
# Feature correlation with target (Purchase)
from pyspark.ml.stat import Correlation

# Correlation matrix
correlation_matrix = Correlation.corr(df_final, "scaled_features").head()

print("\n✓ Correlation matrix computed")
print("(Can be used for feature selection)")

## 16. Summary & Next Steps

In [ ]:
print("\n" + "="*60)
print("✅ FEATURE ENGINEERING PIPELINE COMPLETE")
print("="*60)

print("\n📊 Created Features:")
print("  ✓ User-level aggregations (purchase patterns)")
print("  ✓ Product-level aggregations (popularity, pricing)")
print("  ✓ Category-level aggregations")
print("  ✓ RFM features (Frequency, Monetary)")
print("  ✓ Interaction features (ratios, flags)")
print("  ✓ Categorical encoding (StringIndexer + OneHot)")
print("  ✓ Feature scaling (StandardScaler)")

print("\n💾 Saved Artifacts:")
print("  ✓ Main features dataset → data/processed/delta/features/")
print("  ✓ User features → data/processed/delta/user_features/")
print("  ✓ Product features → data/processed/delta/product_features/")
print("  ✓ Encoding pipeline → models/encoding_pipeline/")
print("  ✓ Scaler model → models/scaler_model/")

print("\n🎯 Next Steps:")
print("  1. Build ETL pipeline (automation)")
print("  2. Train ML models (regression, clustering, recommendation)")
print("  3. Model evaluation and tuning")
print("  4. Setup Kafka + Streaming pipeline")

print("\n" + "="*60)

In [ ]:
# Stop Spark session
# spark.stop()
# print("\n✓ Spark session stopped")